<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/recommend/nobuy_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 32.8 MB/s eta 0:00:00


In [10]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

In [3]:
cart_purchase=pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/cart_purchase.parquet')
customer_info=pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/customer_info.parquet')
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product.parquet')

In [39]:
product_c =product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)

# 2. 데이터 합치기

In [4]:
cart_purchase.head()

,created_at,customer_id,product_id,p_idx,event,c_idx
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968


In [5]:
customer_info.head()

,customer_id,gender,birthdate,c_idx
2,7279,M,1989-01-23,3777
4,82542,M,2000-07-15,41805
7,96453,F,2005-02-12,48889
8,8031,F,1995-11-01,4179
12,9152,F,1995-12-10,4728


In [18]:
customer_info.birthdate=customer_info.birthdate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [22]:
customer_info['year']=customer_info.birthdate.apply(lambda x: datetime.today().year-x.year)

In [23]:
customer_info.head()

,customer_id,gender,birthdate,c_idx,year
2,7279,M,1989-01-23,3777,34
4,82542,M,2000-07-15,41805,23
7,96453,F,2005-02-12,48889,18
8,8031,F,1995-11-01,4179,28
12,9152,F,1995-12-10,4728,28


In [24]:
customer_info['year'].describe()

count    100000.000000
mean         27.516130
std           7.275733
min           7.000000
25%          22.000000
50%          27.000000
75%          32.000000
max          69.000000
Name: year, dtype: float64

In [27]:
df = pd.merge(cart_purchase, customer_info, on=['c_idx','customer_id'], how='left')
df.head()

,created_at,customer_id,product_id,p_idx,event,c_idx,gender,birthdate,year
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039,F,2000-08-20,23
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968,M,2000-12-17,23
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968,M,2000-12-17,23
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968,M,2000-12-17,23
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968,M,2000-12-17,23


In [28]:
df['year'].describe()

count    1.876781e+06
mean     2.736542e+01
std      7.271023e+00
min      7.000000e+00
25%      2.200000e+01
50%      2.700000e+01
75%      3.200000e+01
max      6.900000e+01
Name: year, dtype: float64

# 데이터 분리
- 20대 이하
- 20대
- 30대 이상

In [29]:
youth = df.query("year<20")
age_20 = df.query('year>=20 and year<30')
age_30 = df.query('year>=30')

- 나이 별 추천아이템

In [44]:
item=youth.groupby('p_idx').count().sort_values(by='created_at').tail(25).index
item

Int64Index([16319, 14056, 11862,  5244, 10702, 15287,  5355, 15841, 10691,
            24182,  5259, 30254, 15197,  5477,  5278, 25016,  2580, 25021,
            10714, 16314,  5280, 17273, 24194, 17291,  5246],
           dtype='int64', name='p_idx')

In [37]:
age_20.groupby('p_idx').count().sort_values(by='created_at').tail(25).index

Int64Index([14056, 25049, 16317,  5355,  5244, 10702, 15287,  5259, 24182,
            15841, 30254,  5477, 15197, 10691,  5278, 10714, 25021,  2580,
            25016, 16314,  5280, 24194, 17273, 17291,  5246],
           dtype='int64', name='p_idx')

In [38]:
age_30.groupby('p_idx').count().sort_values(by='created_at').tail(25).index

Int64Index([11861, 25049, 10702, 16319, 30243,  5244, 15287, 24182, 30254,
            15197,  5278, 10691,  5259,  5477, 15841, 25016, 10714, 25021,
             2580,  5280, 17273, 24194, 16314, 17291,  5246],
           dtype='int64', name='p_idx')

# 세대별 추천하기

In [50]:
def nobuy_recommend(c_idx,k=25):
  if c_idx not in customer_info['c_idx']: #정보가 없는 사람에게
    item = cart_purchase.groupby('p_idx').count()['created_at'].sort_values().tail(k).index
    recommend= product_c[product_c['p_idx'].isin(item)]

  elif c_idx in youth: #20세 미만
    item = youth.groupby('p_idx').count().sort_values(by='created_at').tail(k).index
    recommend= product_c[product_c['p_idx'].isin(item)]
  
  elif c_idx in age_20: #20대
    item = age_20.groupby('p_idx').count().sort_values(by='created_at').tail(k).index
    recommend= product_c[product_c['p_idx'].isin(item)]
  
  else: #30대 이상
    item = age_30.groupby('p_idx').count().sort_values(by='created_at').tail(k).index
    recommend= product_c[product_c['p_idx'].isin(item)]

  return recommend

In [51]:
nobuy_recommend(60000,25)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
54,59435,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Arrow Men Black Formal Shoes,2580
76,54118,Women,Footwear,Shoes,Heels,Black,Winter,2012,Casual,Rocia Women Black Flats,25016
123,49005,Women,Accessories,Jewellery,Ring,Silver,Summer,2012,Casual,Lucera Women Silver Ring,16319
810,47191,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Franco Leone Men Black Formal Shoes,10691
948,7390,Men,Footwear,Shoes,Formal Shoes,Brown,Winter,2018,Formal,Red Tape Men Brown Shoes,24194
1110,3398,Women,Accessories,Bags,Handbags,Black,Winter,2015,Casual,Murcia Women Black Handbag,17273
1136,45407,Women,Footwear,Shoes,Flats,Brown,Summer,2012,Casual,Rocia Women Brown Sandals,25049
1249,30692,Women,Footwear,Shoes,Heels,Brown,Winter,2015,Casual,Catwalk Women Brown Heels,5280
1370,48728,Women,Accessories,Jewellery,Earrings,Silver,Summer,2015,Casual,Lucera Women Silver Earrings,16314
1424,47794,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Murcia Women Brown Handbag,17291


## 전체 베스트 셀러

In [54]:
cart_purchase.groupby('p_idx').count()['created_at'].sort_values().tail(25).index

Int64Index([30243, 16317, 25049,  5355, 10702,  5244, 15287, 24182,  5259,
            30254, 15841, 15197, 10691,  5477,  5278, 10714, 25016, 25021,
             2580,  5280, 16314, 24194, 17273, 17291,  5246],
           dtype='int64', name='p_idx')

In [55]:
def best_seller(c_idx,k=25):
  if c_idx not in customer_info['c_idx']: 
    item = cart_purchase.groupby('p_idx').count()['created_at'].sort_values().tail(k).index
    recommend= product_c[product_c['p_idx'].isin(item)]

  return recommend

In [57]:
best_seller(60000000)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
54,59435,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Arrow Men Black Formal Shoes,2580
76,54118,Women,Footwear,Shoes,Heels,Black,Winter,2012,Casual,Rocia Women Black Flats,25016
85,48781,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012,Casual,Lucera Women Silver Pendant,16317
810,47191,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Franco Leone Men Black Formal Shoes,10691
948,7390,Men,Footwear,Shoes,Formal Shoes,Brown,Winter,2018,Formal,Red Tape Men Brown Shoes,24194
1110,3398,Women,Accessories,Bags,Handbags,Black,Winter,2015,Casual,Murcia Women Black Handbag,17273
1136,45407,Women,Footwear,Shoes,Flats,Brown,Summer,2012,Casual,Rocia Women Brown Sandals,25049
1249,30692,Women,Footwear,Shoes,Heels,Brown,Winter,2015,Casual,Catwalk Women Brown Heels,5280
1370,48728,Women,Accessories,Jewellery,Earrings,Silver,Summer,2015,Casual,Lucera Women Silver Earrings,16314
1424,47794,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Murcia Women Brown Handbag,17291
